In [1]:
from main import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stas_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stas_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Сгенерируем синтетический датасет

In [2]:
text1 = """Эта книга адресована всем, кто изучает русский язык."""

text2 = """Воспитанные люди должны удовлетворять следующим условиям."""

text3 = """Когда человек сознательно или интуитивно выбирает себе в жизни какую-то цель, жизненную задачу, он невольно дает себе оценку. По тому, ради чего человек живет, можно судить и о его самооценке - низкой или высокой."""

text4 = """Можно по-разному определять цель своего существования, но цель должна быть. Надо иметь и принципы в жизни. Одно правило в жизни должно быть у каждого человека, в его цели жизни, в его принципах жизни, в его поведении: надо прожить жизнь с достоинством, чтобы не стыдно было вспоминать."""

In [3]:
dataset = {
    'id': [i for i in range(100)],
    'expired_at': ['2021-12-15 11:25:00'] * 100,
    'created_at': ['2021-12-15 11:25:00'] * 100,
    'published_at': ['2021-12-15 11:25:00'] * 100,
    'snapshot_date': ['2021-12-15 11:25:00'] * 100,
    'category_id': ['category_1', 'category_2', 'category_3', 'category_4'] * 25,
    'condition': ['NEW', 'USED', 'NEW', 'No'] * 25,
    'price': np.random.uniform(100, 15000, 100),
    'anonymized_user_id': ['anonymized_user_id_' + str(i) for i in range(25)] * 4,
    'origin': ['FEED', 'FEED', 'FEED', 'FORM'] * 25,
    'status': ['Active'] * 100,
    'title': [text1, text2, text3, text4] * 25,
    'description': [text1, text2, text3, text4] * 25,
    'photo_hash': ['NFJNVJKSJNCKSDNJK', 'JNSCMLKMVJHFBGFVD'] * 50,
    'latitude': [87.87, 87.90] * 50,
    'longitude': [25.25, 25.28] * 50,
    'region_id': [98] * 100,
    'region_name': ['SP'] * 100,
    'preferred_way_to_contact': ['PHONE', 'PHONE', 'No', 'CHAT'] * 25,
}

In [4]:
dataset = pd.DataFrame(dataset)

## Теперь, применим наш Pipeline

In [5]:
_ohe_features = [
    'condition',
    'preferred_way_to_contact',
]

_le_features = [
    'origin',
]

_num_features = [
    'price',
    'latitude',
    'longitude',
]

_tf_idf_features = [
    'title',
    'description',
]

_delete_features = [
     'id',
     'expired_at',
     'created_at',
     'published_at',
     'snapshot_date',
     'category_id',
     'anonymized_user_id',
     'status',
     'photo_hash',
     'region_id',
     'region_name',
]

pipeline = MyPipeline(
    steps=[
        ('text_preprocessor', UpdateTextFeatures(_tf_idf_features).with_steps(
                steps=[
                    StopWordsRemover(),
                    Lemmatizer(),
                    MyTfIdfVectorizer(),
                ]
            )
        ),
        ('feature_generation', FeatureGenerator(_ohe_features, _le_features, _num_features, _delete_features)),
        ('clusterization', Clusterizer())
    ]
)

In [6]:
pred = pipeline.fit_transform(dataset)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
get_duplicate_from_prediction(pred, dataset)

,id1,id2
0,10,34
1,10,86
2,51,59
3,9,41
4,9,49
5,9,65
6,12,48
7,12,56
8,12,68
9,46,58


## Теперь у нас есть ID пар дупликатов, которые мы в будущем будем матчить.